In [2]:
import os
import random
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import time
import helper as helper_file
import matplotlib.pyplot as plt
import shutil 
import os
import shutil
import pandas as pd
import re



# Define paths for pooled dataset
pooled_images_dir = "/home/jayanth/ConsultingProject/DATA/pooled_images"
pooled_labels_dir = "/home/jayanth/ConsultingProject/DATA/pooled_labels"
mismatch_report_path = "/home/jayanth/ConsultingProject/DATA/pooled_data_mismatches.csv"

# Source image and label folders
image_folders = [
   # "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/1_Vocal cords - Copy",
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/1_Vocal cords - Copy_March",
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/2_stomach photos - Copy",
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/3_Proper ett depth - Copy",
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/4_Trachea 1 - Copy"
]

label_folders = [
    #"/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/1_labels_vocal-cords_2025-01-06-09-56-40",
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/5_labels_my-project-name_2025-02-25-12-17-28_vocal_cords_2", 
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/2_labels_my-project-name_2025-01-09-10-07-16",
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/3_labels_my-project-name_2025-02-01-09-53-12 ett",
    "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/4_labels_my-project-name_2025-02-01-04-41-53 trachea"
]


#Define paths for output dataset and YAML files
images_folder = pooled_images_dir
labels_folder = pooled_labels_dir
output_dataset_path = "/home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset"
data_yaml_path = os.path.join('/home/jayanth/ConsultingProject/DATA/output_dataset', "dataset.yaml")
holdout_yaml_path = os.path.join('/home/jayanth/ConsultingProject/DATA/output_dataset', "holdout.yaml")

#tracheal rings and placement lables
tracheal_rings_file = '/home/jayanth/ConsultingProject/DATA/tracheal rings numbered.xlsx'

# Parameters
target_size = (416, 416)
holdout_ratio = 0.10  # ✅ 10% of dataset for holdout (adjustable)

#Tuning model output location 
experiment_logs = "YOLO_model_experiment_logs"



# Combine All Datasets and Analyze missing data

### **🔹 Overview**  
This script **creates a pooled dataset** by merging images and labels from multiple folders, handling duplicates, and generating a mismatch report for missing files.

#### **🔹 Key Functions of the Code**  

#### 1️⃣ Cleaning & Preparing the Dataset
- **Deletes existing pooled dataset folders** (`pooled_images_dir`, `pooled_labels_dir`) to avoid duplicates.  
- **Creates new directories** for storing pooled images and labels.

#### 2️⃣ Copying & Handling Files
- **Finds all images and labels** in the source folders.  
- **Handles duplicate filenames** (e.g., renames duplicates using `_dupX`).  
- **Copies images and labels** into the pooled dataset.

#### 3️⃣ Identifying Mismatches
- **Finds images that don’t have labels.**  
- **Finds labels that don’t have images.**  
- **Stores mismatch details** in a report (`pooled_data_mismatches.csv`).

#### 4️⃣ Printing Dataset Statistics 
- 📂 **Total number of pooled images and labels**  
- ⚠ **Number of images without labels**  
- ⚠ **Number of labels without images**  
- 📁 **Saves a detailed mismatch report for debugging.**


In [3]:
# import os
# import shutil
# import pandas as pd
# import re

# # Define paths for pooled dataset
# pooled_images_dir = "/home/jayanth/ConsultingProject/DATA/pooled_images"
# pooled_labels_dir = "/home/jayanth/ConsultingProject/DATA/pooled_labels"
# mismatch_report_path = "/home/jayanth/ConsultingProject/DATA/pooled_data_mismatches.csv"

# # Source image and label folders
# image_folders = [
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/1_Vocal cords - Copy",
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/2_stomach photos - Copy",
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/3_Proper ett depth - Copy",
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Images/4_Trachea 1 - Copy"
# ]

# label_folders = [
#     #"/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/1_labels_vocal-cords_2025-01-06-09-56-40",
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/5_labels_my-project-name_2025-02-25-12-17-28_vocal_cords_2", 
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/2_labels_my-project-name_2025-01-09-10-07-16",
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/3_labels_my-project-name_2025-02-01-09-53-12 ett",
#     "/home/jayanth/ConsultingProject/DATA/Pooled_Data/Labels/4_labels_my-project-name_2025-02-01-04-41-53 trachea"
# ]

# Remove the existing dataset folder if it exists
if os.path.exists(pooled_images_dir):
    print(f"⚠ Deleting existing dataset folder: {pooled_images_dir}")
    shutil.rmtree(pooled_images_dir)  # Deletes the entire folder
if os.path.exists(pooled_labels_dir):
    print(f"⚠ Deleting existing dataset folder: {pooled_labels_dir}")
    shutil.rmtree(pooled_labels_dir)  # Deletes the entire folder


# Create pooled directories if they don’t exist
os.makedirs(pooled_images_dir, exist_ok=True)
os.makedirs(pooled_labels_dir, exist_ok=True)

# Function to clean filenames (remove _dupX suffix)
def clean_filename(filename):
    return re.sub(r'_dup\d+', '', filename)  # Removes _dup1, _dup2, etc.

# Function to count and map filenames to folders
def count_files_and_map(source_folders, file_types):
    file_to_folder = {}
    folder_counts = {}
    
    for folder in source_folders:
        if not os.path.exists(folder):
            print(f"⚠ Warning: Folder {folder} does not exist. Skipping.")
            folder_counts[folder] = 0
            continue
        
        files = [f for f in os.listdir(folder) if os.path.splitext(f)[1].lower() in file_types]
        folder_counts[folder] = len(files)
        
        for f in files:
            name, _ = os.path.splitext(f)
            clean_name = clean_filename(name)  # Remove duplicates suffixes
            
            if clean_name in file_to_folder:
                file_to_folder[clean_name].append(folder)
            else:
                file_to_folder[clean_name] = [folder]

    return file_to_folder, folder_counts

# Get original locations and counts of images and labels
image_to_folder, image_folder_counts = count_files_and_map(image_folders, {".jpg", ".png", ".jpeg"})
label_to_folder, label_folder_counts = count_files_and_map(label_folders, {".xml"})

# Function to copy files and track duplicates
def copy_files_with_handling(source_folders, destination_folder):
    existing_files = set()
    for folder in source_folders:
        if not os.path.exists(folder):
            continue

        for filename in os.listdir(folder):
            name, ext = os.path.splitext(filename)
            if ext.lower() not in {".jpg", ".png", ".jpeg", ".xml"}:
                continue  # Skip unsupported files

            source_path = os.path.join(folder, filename)
            dest_path = os.path.join(destination_folder, filename)

            # Handle duplicate filenames
            counter = 1
            while os.path.exists(dest_path):
                new_filename = f"{name}_dup{counter}{ext}"
                dest_path = os.path.join(destination_folder, new_filename)
                counter += 1

            # Copy file to the pooled directory
            shutil.copy2(source_path, dest_path)
            existing_files.add(clean_filename(name))  # Store cleaned filename

    return existing_files

# Copy images and track total pooled count
pooled_images = copy_files_with_handling(image_folders, pooled_images_dir)
pooled_labels = copy_files_with_handling(label_folders, pooled_labels_dir)

# Count total files in pooled folders
total_pooled_images = len(os.listdir(pooled_images_dir))
total_pooled_labels = len(os.listdir(pooled_labels_dir))

# Find mismatches (after cleaning filenames)
images_without_labels = pooled_images - pooled_labels
labels_without_images = pooled_labels - pooled_images

# Prepare data for mismatch CSV
mismatch_data = []

# Add missing images with their original folder
for img in images_without_labels:
    mismatch_data.append({
        "Filename": img,
        "Type": "Image without Label",
        "Source Folder": ", ".join(image_to_folder.get(img, ["Unknown"]))
    })

# Add missing labels with their original folder
for lbl in labels_without_images:
    mismatch_data.append({
        "Filename": lbl,
        "Type": "Label without Image",
        "Source Folder": ", ".join(label_to_folder.get(lbl, ["Unknown"]))
    })

# Save mismatch report to CSV
df_mismatches = pd.DataFrame(mismatch_data)
df_mismatches.to_csv(mismatch_report_path, index=False)

# Print summary of counts
print("\n📊 **Folder-wise Breakdown**")
print("\n📂 **Images Per Folder:**")
for folder, count in image_folder_counts.items():
    print(f"   - {folder.split('/')[-1]}: {count} images")

print("\n📂 **Labels Per Folder:**")
for folder, count in label_folder_counts.items():
    print(f"   - {folder.split('/')[-1]}: {count} labels")

print("\n✅ **Pooled Dataset Summary**")
print(f"📂 Total Pooled Images: {total_pooled_images}")
print(f"📂 Total Pooled Labels: {total_pooled_labels}")
print(f"⚠ Images without Labels: {len(images_without_labels)}")
print(f"⚠ Labels without Images: {len(labels_without_images)}")
print(f"\n📁 Mismatch report saved at: {mismatch_report_path}")

⚠ Deleting existing dataset folder: /home/jayanth/ConsultingProject/DATA/pooled_images
⚠ Deleting existing dataset folder: /home/jayanth/ConsultingProject/DATA/pooled_labels

📊 **Folder-wise Breakdown**

📂 **Images Per Folder:**
   - 1_Vocal cords - Copy_March: 2707 images
   - 2_stomach photos - Copy: 748 images
   - 3_Proper ett depth - Copy: 293 images
   - 4_Trachea 1 - Copy: 635 images

📂 **Labels Per Folder:**
   - 5_labels_my-project-name_2025-02-25-12-17-28_vocal_cords_2: 2700 labels
   - 2_labels_my-project-name_2025-01-09-10-07-16: 748 labels
   - 3_labels_my-project-name_2025-02-01-09-53-12 ett: 289 labels
   - 4_labels_my-project-name_2025-02-01-04-41-53 trachea: 611 labels

✅ **Pooled Dataset Summary**
📂 Total Pooled Images: 4383
📂 Total Pooled Labels: 4348
⚠ Images without Labels: 35
⚠ Labels without Images: 0

📁 Mismatch report saved at: /home/jayanth/ConsultingProject/DATA/pooled_data_mismatches.csv


# Data Prep - Creating Train Val and Holdout Datasets 
### **🔹 Overview**  
This script **processes the pooled dataset** by splitting images into training, validation, and holdout sets, handling class label modifications, and generating YOLO-compatible datasets.

#### **🔹 Key Functions of the Code**  

#### 1️⃣ Cleaning & Preparing the Dataset  
- **Deletes existing dataset folders** (`output_dataset_path`) to avoid duplication.  
- **Creates new directories** for `train`, `val`, and `holdout` datasets.

#### 2️⃣ Processing & Splitting the Dataset
- **Extracts unique labels from the dataset** and creates a class mapping.  
- **Splits images into train (80%), validation (20%), and holdout (10%)**  
- **Ensures holdout set is balanced** by selecting images proportionally from each class.  

#### 3️⃣ Handling Tracheal Rings Labels
- **Loads `tracheal rings numbered.xlsx`** to map images to tracheal ring categories.  
- **Replaces `placement` labels** with `tracheal_rings_3`, `tracheal_rings_4`, `tracheal_rings_5`, or `tracheal_rings_greater_than_5`.  
- **Updates `class_mapping`** after modifying labels.

#### 4️⃣ Generating YOLO-Compatible Dataset
- **Processes images and labels**, converts them into YOLO format.  
- **Saves processed images and labels** to `train`, `val`, and `holdout` folders.  
- **Generates `dataset.yaml` and `holdout.yaml`** for YOLO training.

#### 5️⃣ Dataset Validation & Class Distribution Analysis
- **Validates the dataset** to check for missing labels or images.  
- **Counts the number of images per class** after label replacement.  
- **Prints a summary of dataset statistics**.


In [4]:
# import os
# import random
# import cv2
# import numpy as np
# import xml.etree.ElementTree as ET
# import time
# import helper as helper_file
# import matplotlib.pyplot as plt
# import shutil 

# images_folder = pooled_images_dir
# labels_folder = pooled_labels_dir
# output_dataset_path = "/home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset"
# data_yaml_path = os.path.join('/home/jayanth/ConsultingProject/DATA/output_dataset', "dataset.yaml")
# holdout_yaml_path = os.path.join('/home/jayanth/ConsultingProject/DATA/output_dataset', "holdout.yaml")

# #tracheal rings and placement lables
# tracheal_rings_file = '/home/jayanth/ConsultingProject/DATA/tracheal rings numbered.xlsx'

# # Parameters
# target_size = (416, 416)
# holdout_ratio = 0.10  # ✅ 10% of dataset for holdout (adjustable)

# Remove the existing dataset folder if it exists
if os.path.exists(output_dataset_path):
    print(f"⚠ Deleting existing dataset folder: {output_dataset_path}")
    shutil.rmtree(output_dataset_path)  # Deletes the entire folder

#Common code for all folders
# Create necessary directories - dont create if already exists. 
os.makedirs(output_dataset_path, exist_ok=True)
os.makedirs(os.path.join(output_dataset_path, "train/images"), exist_ok=True)
os.makedirs(os.path.join(output_dataset_path, "train/labels"), exist_ok=True)
os.makedirs(os.path.join(output_dataset_path, "val/images"), exist_ok=True)
os.makedirs(os.path.join(output_dataset_path, "val/labels"), exist_ok=True)
os.makedirs(os.path.join(output_dataset_path, "holdout/images"), exist_ok=True)
os.makedirs(os.path.join(output_dataset_path, "holdout/labels"), exist_ok=True)


# Validate train and validation datasets
train_images_folder = output_dataset_path +"/train/images"
train_labels_folder = output_dataset_path + "/train/labels"
val_images_folder = output_dataset_path + "/val/images"
val_labels_folder = output_dataset_path +"/val/labels"
holdout_images_folder = output_dataset_path + "/holdout/images"
holdout_labels_folder = output_dataset_path +"/holdout/labels"



print('-------------------------------------Starting to process new dataset---------------------------------------------')

print(images_folder, labels_folder)
# Extract unique labels and create a class mapping
print("Extracting unique labels from the dataset...")
unique_labels = sorted(helper_file.extract_unique_labels(labels_folder, combine_right_left_flag = True ))  # Sort for consistent ordering
class_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print(f"Class mapping created: {class_mapping}")


# Split data into train/val/holdout
# Step 1: Get all image files
image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

# Step 2: Parse XML labels to identify class distribution
image_class_map = {}
for image_name in image_files:
    label_path = os.path.join(labels_folder, os.path.splitext(image_name)[0] + ".xml")
    if os.path.exists(label_path):
        try:
            _, labels = helper_file.parse_xml(label_path, combine_right_left_flag = True )  # Extract class labels from XML
            if labels:
                image_class_map[image_name] = labels  # Store image and its class labels
        except Exception as e:
            print(f"Skipping problematic file {label_path}: {e}")

# Step 3: Organize images by class
class_images = {cls: [] for cls in class_mapping.keys()}
for image, labels in image_class_map.items():
    for label in labels:
        if label in class_images:
            class_images[label].append(image)

# Step 4: Select Holdout Set Proportionally From Each Class
holdout_images = []
for cls, images in class_images.items():
    num_holdout_class = int(len(images) * holdout_ratio)  # Select 10% from each class
    holdout_images.extend(random.sample(images, min(num_holdout_class, len(images))))  # Ensure enough samples

# Step 5: Remove Holdout Images & Split Remaining into Train/Val
remaining_images = list(set(image_files) - set(holdout_images))
random.shuffle(remaining_images)
# train_images = remaining_images[:int(0.8 * len(remaining_images))]
# val_images = remaining_images[int(0.2 * len(remaining_images)):]
train_split = int(0.8 * len(remaining_images))
train_images = remaining_images[:train_split]
val_images = remaining_images[train_split:]  # ✅ Take the last 20%


print(f"Final Dataset Split - Train: {len(train_images)}, Val: {len(val_images)}, Holdout: {len(holdout_images)}")


# Process and save data
problematic_files = []  # List to store names of problematic XML files

#tracheal rings data processing and replacing the placement label 
#Create image_to_class for tracheal rings classes 
# Create image_to_class for tracheal rings classes 
image_to_class_trachea = {}
df = pd.read_excel(tracheal_rings_file)

for _, row in df.iterrows():
    image_name = str(row['image_file_name']).strip() + ".jpg"  # ✅ Add .jpg to match image_class_map
    rings_count = row['tracheal_rings']
    
    if rings_count == 3:
        image_to_class_trachea[image_name] = 'tracheal_rings_3'
    elif rings_count == 4:
        image_to_class_trachea[image_name] = 'tracheal_rings_4'
    elif rings_count == 5:
        image_to_class_trachea[image_name] = 'tracheal_rings_5'
    elif rings_count == 'greater than 5':
        image_to_class_trachea[image_name] = 'tracheal_rings_greater_than_5'

# Step 1: Process all labels and replace 'placement'
processed_labels = set()
for image_name, labels in image_class_map.items():
    for label in labels:
        if label == "placement" and image_name in image_to_class_trachea:
            processed_labels.add(image_to_class_trachea[image_name])  # Add tracheal rings class
        else:
            processed_labels.add(label)  # Add other labels

# Step 2: Create class mapping AFTER processing labels
print("Creating class mapping after label replacement...")
class_mapping = {label: idx for idx, label in enumerate(sorted(processed_labels))}
print(f"✅ Updated Class Mapping: {class_mapping}")



#Creating train test and holdout datasets 
for split, split_images in [("train", train_images), ("val", val_images), ("holdout", holdout_images)]:
    print(f"\nProcessing {split} set with {len(split_images)} images...")
    for image_name in split_images:
        image_path = os.path.join(images_folder, image_name)
        label_path = os.path.join(labels_folder, os.path.splitext(image_name)[0] + ".xml")
        if os.path.exists(label_path):
            try:
                bboxes, labels = helper_file.parse_xml(label_path, combine_right_left_flag = True )

                # Replace 'placement' with the correct tracheal rings class
                updated_labels = []
                for label in labels:
                    if label == 'placement' and image_name in image_to_class_trachea:
                        updated_labels.append(image_to_class_trachea[image_name])  # Assign tracheal_rings_X label
                    else:
                        updated_labels.append(label)  # Keep other labels unchanged

                # Step 2: Store updated labels in image_class_map
                image_class_map[image_name] = updated_labels  # ✅ Ensure placement is fully replaced



                if bboxes:
                    image, scale, top, left = helper_file.preprocess_image(image_path, target_size)
                    yolo_bboxes = helper_file.convert_to_yolo_format(class_mapping, bboxes, updated_labels, target_size, scale, top, left)
                    helper_file.save_yolo_data(output_dataset_path,split, image_name, image, yolo_bboxes)
                else:
                    problematic_files.append(label_path)
            except Exception as e:
                print(f"Skipping problematic file {label_path}: {e}")
                problematic_files.append(label_path)


# Create YOLO dataset YAML file
print("\nCreating dataset YAML file...")

with open(data_yaml_path, "w") as f:
    f.write(f"""
train: {os.path.join(output_dataset_path, 'train/images')}
val: {os.path.join(output_dataset_path, 'val/images')}
holdout: {os.path.join(output_dataset_path, 'holdout/images')}
nc: {len(class_mapping)}
names: {list(class_mapping.keys())}
""")
print("Dataset YAML file created successfully!")


with open(holdout_yaml_path, "w") as f:
    f.write(f"""
train: {os.path.join(output_dataset_path, 'train/images')}
val: {os.path.join(output_dataset_path, 'holdout/images')}
nc: {len(class_mapping)}
names: {list(class_mapping.keys())}
""")
print("Dataset YAML file created successfully!")


# # Validate train and validation datasets
# train_images_folder = output_dataset_path +"/train/images"
# train_labels_folder = output_dataset_path + "/train/labels"
# val_images_folder = output_dataset_path + "/val/images"
# val_labels_folder = output_dataset_path +"/val/labels"
# holdout_images_folder = output_dataset_path + "/holdout/images"
# holdout_labels_folder = output_dataset_path +"/holdout/labels"


print("Validating train dataset...")
train_problems = helper_file.validate_dataset(train_images_folder, train_labels_folder)
print("\nValidating validation dataset...")
val_problems = helper_file.validate_dataset(val_images_folder, val_labels_folder)

# Report problematic files
if train_problems or val_problems:
    print("\nProblematic files found:")
    for problem in train_problems + val_problems:
        print(problem)
else:
    print("\nAll files are valid!")

print('-------------------------------------Ended processing one dataset---------------------------------------------')

print("Training Images Count :" , len([f for f in os.listdir(train_images_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]))
print("Validation Images Count :" , len([f for f in os.listdir(val_images_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]))
print("Holdout Images Count :" , len([f for f in os.listdir(holdout_images_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]))
print(class_mapping)

#Printing the class distribution 
# Step 1: Initialize a dictionary to count occurrences per class
class_counts = {label: 0 for label in class_mapping.keys()}

# Step 2: Count occurrences in the updated image_class_map
for image_name, labels in image_class_map.items():  # ✅ Use updated labels
    for label in labels:
        if label in class_counts:
            class_counts[label] += 1

# Step 3: Print class distribution
print("\n📊 **Final Class Distribution**")
for label, count in class_counts.items():
    print(f"{label}: {count} images")





⚠ Deleting existing dataset folder: /home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset
-------------------------------------Starting to process new dataset---------------------------------------------
/home/jayanth/ConsultingProject/DATA/pooled_images /home/jayanth/ConsultingProject/DATA/pooled_labels
Extracting unique labels from the dataset...
Error reading XML file /home/jayanth/ConsultingProject/DATA/pooled_labels/Vvvv&uquu.xml: not well-formed (invalid token): line 3, column 24
Error reading XML file /home/jayanth/ConsultingProject/DATA/pooled_labels/Whahhshh&h.xml: not well-formed (invalid token): line 3, column 25
Error reading XML file /home/jayanth/ConsultingProject/DATA/pooled_labels/H&ygfxc.xml: not well-formed (invalid token): line 3, column 22
Error reading XML file /home/jayanth/ConsultingProject/DATA/pooled_labels/5ghyuh&u.xml: not well-formed (invalid token): line 3, column 23
Class mapping created: {'Arytenoid': 0, 'Epigottis': 1, 'Esophagus': 2, 'Glottic

# 🚀 YOLOv8 Model Training & Tuning Pipeline

## 📌 Overview
This script **trains, evaluates, and fine-tunes** YOLOv8 models on a custom dataset.  
It includes **experiment logging, model validation, mobile conversion, and inference visualization**.


#### 🔹 1. Setup & Data Preparation
- **Checks if the dataset exists** before proceeding.
- **Creates an experiment logging folder** (`YOLO_model_experiment_logs`) to store training results.



#### 🔹 2. Define YOLO Experiments & Hyperparameters
- Configures different **YOLO model versions** (e.g., `yolov8s.pt`, `yolov8m.pt`).
- Defines **batch size, image size, learning rate, and epochs** for each experiment.



#### 🔹 3. Model Training & Validation
- Iterates through **multiple hyperparameter configurations**.
- **Trains the YOLO model** using the dataset (`dataset.yaml`).
- **Validates the model** on the **holdout set** (`holdout.yaml`).



#### 🔹 4. Performance Logging & Evaluation
- **Extracts global metrics** (Precision, Recall, mAP50, mAP50-95).
- **Extracts per-class metrics** and stores them in a CSV log file.
- Saves the **trained model (`trained_model.pt`)** in the respective experiment folder.


#### 🔹 5. Model Conversion for Mobile Deployment
- Converts the trained model into a **mobile-compatible format** using `helper_file.mobile_optimized_model_conversion()`.



#### 🔹 6. Inference & Visualization
- Runs inference on **random holdout images**.
- Draws **Ground Truth (GT) bounding boxes**.
- Plots **YOLOv8 Predictions vs. GT** side by side.
- Saves comparison **visualization plots** in the `Results_comparison_plots` folder.



#### ✅ Final Output
- **Trained YOLO models** stored in `YOLO_model_experiment_logs/{experiment_name}/trained_model.pt`.
- **Performance logs** stored in `yolo_experiments.csv`.
- **Mobile-optimized models** stored in the same directory.
- **Inference results & comparison plots** saved for easy evaluation.

🚀 **This pipeline automates YOLOv8 training, logging, validation, and optimization for real-world deployment!** 🔥


In [4]:
output_dataset_path

'/home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset'

In [5]:
#---------------------------------------------------- MODEL TRAINING & TUNING ------------------------------------------------------------------------

import os
import cv2
import numpy as np
import time
import torch
import pandas as pd
import random
import matplotlib.pyplot as plt
from ultralytics import YOLO
import helper as helper_file

# # # Paths to dataset & logs
# output_dataset_path = "/home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset"
# data_yaml_path = os.path.join('/home/jayanth/ConsultingProject/DATA/output_dataset', "dataset.yaml")
# holdout_yaml_path = os.path.join('/home/jayanth/ConsultingProject/DATA/output_dataset', "holdout.yaml")
# experiment_logs = "YOLO_model_experiment_logs"

# Ensure dataset exists
if not os.path.exists(output_dataset_path):
    raise FileNotFoundError("Training Data is missing!")

#Create the experiments folder if it does not exist 
os.makedirs(experiment_logs, exist_ok=True)

# Load Holdout Set
holdout_images_folder = os.path.join(output_dataset_path, "holdout/images")
holdout_labels_folder = os.path.join(output_dataset_path, "holdout/labels")
holdout_images = [f for f in os.listdir(holdout_images_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

# Function to log results
def log_results(log_file, config, metrics, class_metrics):
    log_data = {**config, **metrics}  # Combine hyperparameters & global metrics

    # Add per-class metrics to the log
    for class_name, class_data in class_metrics.items():
        log_data.update({
            f"{class_name}_P": class_data["P"],
            f"{class_name}_R": class_data["R"],
            f"{class_name}_mAP50": class_data["mAP50"],
            f"{class_name}_mAP50-95": class_data["mAP50-95"]
        })

    # Save results to CSV
    df = pd.DataFrame([log_data])
    df.to_csv(log_file, mode='a', header=not os.path.exists(log_file), index=False)  # Append if exists

# List of YOLO model experiments

experiments = [
    {"model": "yolov8s.pt", "lr0": 0.001, "batch": 32, "imgsz": 416, "epochs": 100},
    # {"model": "yolov8s.pt", "lr0": 0.001, "batch": 32, "imgsz": 640, "epochs": 50},
    # {"model": "yolov8m.pt", "lr0": 0.001, "batch": 16, "imgsz": 416, "epochs": 50},
]

# Run experiments
for i, exp in enumerate(experiments):
    print(f"\n🚀 Running Experiment {i + 1}: {exp}")

    # Create experiment folder
    experiment_name = f"{exp['model'].split('.')[0]}_lr{exp['lr0']}_batch{exp['batch']}_img{exp['imgsz']}_epochs{exp['epochs']}"
    experiment_folder = os.path.join(experiment_logs, experiment_name)

    
    # If logs folder already exists, delete it and overwrite 
    if os.path.exists(experiment_folder):
        print(f"⚠ Deleting existing dataset folder: {experiment_folder}")
        shutil.rmtree(experiment_folder)  # Deletes the entire folder
    os.makedirs(experiment_folder, exist_ok=True)

    # Load YOLO model
    model = YOLO(exp["model"])

    # Train the model
    model.train(
        data=data_yaml_path,
        epochs=exp["epochs"],
        imgsz=exp["imgsz"],
        batch=exp["batch"],
        lr0=exp["lr0"],
        name="training",
        project=experiment_folder
    )

    # Validate the model on the holdout set
    results = model.val(data=holdout_yaml_path, imgsz=exp["imgsz"], name="holdout", project=experiment_folder)

    # Extract global metrics
    metrics = {
        "Precision": results.box.mp,
        "Recall": results.box.mr,
        "mAP50": results.box.map50,
        "mAP50-95": results.box.map,
    }

    # Extract per-class metrics
    class_metrics = {}
    for class_idx, class_name in results.names.items():
        if class_idx >= len(results.box.p):  # Skip missing classes
            print(f"Skipping class {class_name}: not present in results.")
            continue

        # Extract class-specific results
        class_result = results.box.class_result(class_idx)
        class_metrics[class_name] = {
            "P": class_result[0], "R": class_result[1], "mAP50": class_result[2], "mAP50-95": class_result[3]
        }

    # Log results
    log_file = os.path.join(experiment_folder, "yolo_experiments.csv")
    log_results(log_file, exp, metrics, class_metrics)

    # Save trained model
    model_save_path = os.path.join(experiment_folder, "trained_model.pt")
    model.save(model_save_path)
    print(f"✅ Model saved to {model_save_path}")

    
    #converting the current model to mobile compatible model 
    import torch
    dummy_input = torch.rand(1, 3, target_size[0], target_size[1])
    helper_file.mobile_optimized_model_conversion(experiment_folder, model_save_path,dummy_input)

    # Save Prediction Plots
    plot_folder = os.path.join(experiment_folder, "Results_comparison_plots")
    os.makedirs(plot_folder, exist_ok=True)

    # Generate and save inference plots for a subset of holdout images
    for i, image_name in enumerate(random.sample(holdout_images, min(10, len(holdout_images)))):  # Limit to 10 images
        image_path = os.path.join(holdout_images_folder, image_name)
        label_path = os.path.join(holdout_labels_folder, os.path.splitext(image_name)[0] + ".txt")

        # Load image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load ground truth labels
        gt_bboxes = helper_file.read_yolo_labels(label_path) if os.path.exists(label_path) else []

        # Perform inference
        predictions = model.predict(source=image_path, imgsz=exp["imgsz"])

        # Draw ground truth and predictions
        image_with_gt = helper_file.draw_bboxes(image.copy(), gt_bboxes, list(class_metrics.keys()), color=(0, 255, 0))
        image_with_predictions = cv2.cvtColor(predictions[0].plot(), cv2.COLOR_BGR2RGB)

        # Save comparison plots
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.title("Ground Truth")
        plt.imshow(image_with_gt)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title("Predictions")
        plt.imshow(image_with_predictions)
        plt.axis('off')

        save_path = os.path.join(plot_folder, f"plot_{image_name}.png")
        plt.savefig(save_path)
        print(f"📊 Saved inference plot: {save_path}")

print("\n✅ All experiments completed! Results logged.")



🚀 Running Experiment 1: {'model': 'yolov8s.pt', 'lr0': 0.001, 'batch': 32, 'imgsz': 416, 'epochs': 100}
New https://pypi.org/project/ultralytics/8.3.91 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.65 🚀 Python-3.12.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 11980MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/jayanth/ConsultingProject/DATA/output_dataset/dataset.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=YOLO_model_experiment_logs/yolov8s_lr0.001_batch32_img416_epochs100, name=training, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou

E0000 00:00:1742189288.890267   13150 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742189288.892574   13150 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

train: Scanning /home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset/train/labels... 2800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2800/2800 [00:01<00:00, 1489.78it/s]

train: New cache created: /home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset/train/labels.cache


...

                   all        700       1937      0.768      0.825      0.835       0.55
             Arytenoid        281        540      0.888       0.87      0.901      0.466
             Epigottis        197        198      0.869      0.929      0.951      0.687
             Esophagus        128        128      0.916      0.922      0.962      0.476
               Glottis        397        397      0.897      0.914      0.938       0.57
               Trachea        111        111      0.913      0.973      0.979      0.679
           Vocal Cords        273        517      0.823      0.883      0.877        0.5
      tracheal_rings_3         10         10      0.642        0.8      0.887      0.764
      tracheal_rings_4          9          9      0.535      0.667      0.596      0.458
      tracheal_rings_5         19         19      0.719      0.789      0.866      0.658
tracheal_rings_greater_than_5          8          8      0.477        0.5      0.396      0.243
Speed: 0.0ms p

val: Scanning /home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset/holdout/labels... 844 images, 0 backgrounds, 0 corrupt: 100%|██████████| 844/844 [00:00<00:00, 1481.04it/s]

val: New cache created: /home/jayanth/ConsultingProject/DATA/output_dataset/yolo_dataset/holdout/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:03<00:00,  7.57it/s]


                   all        844       3221      0.728      0.733      0.721      0.492
             Arytenoid        577       1130      0.896       0.87      0.912      0.468
             Epigottis        418        420      0.863      0.936       0.96      0.719
             Esophagus         77         77      0.923      0.933      0.923      0.417
       Glottic Opening          1          1          0          0          0          0
               Glottis        662        662      0.904      0.934      0.941      0.544
               Trachea         61         61      0.882      0.869      0.928      0.645
           Vocal Cords        444        840      0.856        0.9      0.918      0.487
             placement          1          1          1          0          0          0
      tracheal_rings_3          9          9      0.591      0.643      0.697      0.627
      tracheal_rings_4          5          5      0.421        0.8      0.772      0.637
      tracheal_rings_

In [6]:
# Extract Mean Average Precision at IoU 0.5:0.95 (best overall metric)
final_map5095 = results.box.map  # mAP50-95

# Print the key model performance metric
print(f"\n🚀 **Final Model Performance:** mAP50-95 = {final_map5095:.4f}")



🚀 **Final Model Performance:** mAP50-95 = 0.4920
